In [3]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from pinecone import Pinecone

# ✅ 반드시 필요 (이 셀에서 .env 로드)
load_dotenv(override=True)

# === 환경변수 읽기 ===
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")

# === 환경변수 검증 (중요!) ===
if not OPENAI_API_KEY:
    raise ValueError("❌ OPENAI_API_KEY가 설정되지 않았습니다.")
if not PINECONE_API_KEY:
    raise ValueError("❌ PINECONE_API_KEY가 설정되지 않았습니다.")
if not PINECONE_INDEX_NAME:
    raise ValueError("❌ PINECONE_INDEX_NAME가 설정되지 않았습니다 (.env 확인).")

# === 클라이언트 생성 ===
client = OpenAI(api_key=OPENAI_API_KEY)
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(PINECONE_INDEX_NAME)

# === 모델 설정 ===
EMBED_MODEL = "text-embedding-3-large"   # 3072-dim
CHAT_MODEL = "gpt-4.1-mini"

print("✅ OpenAI & Pinecone 연결 완료")
print("   - Index:", PINECONE_INDEX_NAME)

✅ OpenAI & Pinecone 연결 완료
   - Index: realestate


In [10]:
# =========================
# RAG CHATBOT (CONTINUATION)
# =========================

# -------------------------
# 1) 임베딩
# -------------------------
def embed(text: str) -> list[float]:
    return client.embeddings.create(
        model=EMBED_MODEL,
        input=text
    ).data[0].embedding


# -------------------------
# 2) Pinecone 검색
# -------------------------
def retrieve(
    query: str,
    namespace: str = "contracts",
    top_k: int = 7,
):
    qvec = embed(query)
    res = index.query(
        vector=qvec,
        top_k=top_k,
        include_metadata=True,
        namespace=namespace,
    )

    contexts = []
    for m in res.matches or []:
        md = m.metadata or {}
        contexts.append({
            "score": m.score,
            "doc_id": md.get("doc_id"),
            "chunk_id": md.get("chunk_id"),
            "text": md.get("text", ""),
        })
    return contexts


# -------------------------
# 3) 최소 대화 메모리
# -------------------------
class ChatMemory:
    def __init__(self, max_turns: int = 3):
        self.max_turns = max_turns
        self.turns = []  # [{"q":..., "a":...}]

    def add(self, q: str, a: str):
        self.turns.append({
            "q": q,
            "a": a[:160]   # 요약만 저장 (비용/환각 방지)
        })
        self.turns = self.turns[-self.max_turns:]

    def brief(self) -> str:
        if not self.turns:
            return "(이전 대화 없음)"
        return "\n".join(
            [f"- Q: {t['q']}\n  A요지: {t['a']}" for t in self.turns]
        )


# -------------------------
# 4) RAG 답변 생성
# -------------------------
SYSTEM = """
- 반드시 근거 문장을 2~3개 따옴표로 직접 인용하라
- 근거 문장에 없는 평가는 하지 마라
- 근거 문장을 재구성하여 결론을 도출하라
- 해석/추론이 필요한 경우 반드시 '근거 부족'이라고 말하라
"""

def answer_with_rag(
    question: str,
    memory: ChatMemory,
    namespace: str = "contracts",
    top_k: int = 7,
):
    contexts = retrieve(question, namespace=namespace, top_k=top_k)

    if not contexts:
        return "근거 부족"

    context_block = "\n\n".join(
        [
            f"[doc={c['doc_id']} chunk={c['chunk_id']} score={c['score']:.3f}]\n{c['text']}"
            for c in contexts
        ]
    )

    prompt = f"""
[이전 대화 요약]
{memory.brief()}

[근거 context]
{context_block}

[질문]
{question}

[답변 규칙]
- 리스크 위주로 정리
- 근거에 없는 내용 금지
"""

    resp = client.chat.completions.create(
        model=CHAT_MODEL,
        messages=[
            {"role": "system", "content": SYSTEM},
            {"role": "user", "content": prompt},
        ]
    )

    answer = resp.choices[0].message.content
    memory.add(question, answer)
    return answer

In [11]:
memory = ChatMemory()

print("🤖 Q1")
print(answer_with_rag(
    "손해배상 한도 조항에서 어떤 리스크가 있어?",
    memory
))

print("\n🤖 Q2 (이전 대화 기반)")
print(answer_with_rag(
    "그 예외 조항은 왜 문제가 되지?",
    memory
))

🤖 Q1
다음과 같은 리스크가 있습니다.

1. 보상 상한 초과 리스크  
   “본 계약의 손해배상 한도는 연간 계약 금액의 100%로 한다.”  
   → 실제 손해액이 연간 계약 금액의 100%를 넘을 경우 초과분에 대해 보상을 받을 수 없습니다.

2. 무제한 책임 리스크  
   “단, 고의 또는 중과실, 비밀유지 위반, 지식재산권 침해의 경우 손해배상 한도를 적용하지 않는다.”  
   → 해당 사유가 발생하면 손해배상 한도 없이 전액 책임을 져야 할 수 있습니다.

🤖 Q2 (이전 대화 기반)
1. 무제한 책임 발생 리스크  
   근거: “단, 고의 또는 중과실, 비밀유지 위반, 지식재산권 침해의 경우 손해배상 한도를 적용하지 않는다.”  
   → 해당 사유만 발생하면 연간 계약 금액 100%라는 상한이 사라져, 이론상 무제한의 손해배상 책임을 부담하게 됩니다.

2. 예외 범위의 광범위·불명확 리스크  
   근거: “고의 또는 중과실, 비밀유지 위반, 지식재산권 침해”  
   → ‘비밀유지 위반’이나 ‘지식재산권 침해’ 등 판단 기준이 구체적으로 정의되지 않아, 사소한 해석 차이만으로도 상한이 제외될 여지가 큽니다.


In [16]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from pinecone import Pinecone

# =========================
# 0) ENV & CLIENT
# =========================
load_dotenv(override=True)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")

if not OPENAI_API_KEY or not PINECONE_API_KEY or not PINECONE_INDEX_NAME:
    raise ValueError("❌ .env 설정 확인 필요")

client = OpenAI(api_key=OPENAI_API_KEY)
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(PINECONE_INDEX_NAME)

EMBED_MODEL = "text-embedding-3-large"
CHAT_MODEL  = "gpt-4.1-mini"   # 비싼 모델 불필요

print("✅ Pinecone 기반 자료 해설형 챗봇 준비 완료")


# =========================
# 1) EMBEDDING
# =========================
def embed(text: str) -> list[float]:
    return client.embeddings.create(
        model=EMBED_MODEL,
        input=text
    ).data[0].embedding


# =========================
# 2) PINECONE에서 '자료' 가져오기
# =========================
def fetch_sources(
    query: str,
    namespace: str = "contracts",
    top_k: int = 8,
):
    qvec = embed(query)
    res = index.query(
        vector=qvec,
        top_k=top_k,
        include_metadata=True,
        namespace=namespace,
    )

    sources = []
    for m in res.matches or []:
        md = m.metadata or {}
        if md.get("text"):
            sources.append({
                "doc_id": md.get("doc_id"),
                "chunk_id": md.get("chunk_id"),
                "text": md.get("text")
            })
    return sources


# =========================
# 3) 자료 해설 전용 SYSTEM PROMPT
# =========================
SYSTEM = """너의 역할은 '자료 해설자'다.

규칙:
- 반드시 제공된 [자료 발췌]만을 근거로 답변하라.
- 자료에 없는 사실, 평가, 일반론, 추정은 절대 추가하지 마라.
- 새로운 리스크를 만들어내지 마라.
- 답변은 반드시 아래 구조를 따른다.

[출력 구조]
1. [자료 발췌] : 제공된 문장을 그대로 인용
2. [자료 정리] : 발췌 문장의 의미를 묶어 정리 (새 정보 금지)
3. [해설] : 위 자료가 의미하는 바를 최소한으로 설명

불확실하거나 자료가 부족하면 반드시 '자료 부족'이라고 말하라.
"""


# =========================
# 4) 자료 기반 해설 챗봇
# =========================
def explain_with_sources(
    question: str,
    namespace: str = "contracts",
    top_k: int = 8,
):
    sources = fetch_sources(question, namespace, top_k)

    if not sources:
        return "자료 부족"

    source_block = "\n".join(
        [
            f'- "{s["text"]}" (doc={s["doc_id"]}, chunk={s["chunk_id"]})'
            for s in sources
        ]
    )

    prompt = f"""
[질문]
{question}

[자료 발췌]
{source_block}
"""

    resp = client.chat.completions.create(
        model=CHAT_MODEL,
        messages=[
            {"role": "system", "content": SYSTEM},
            {"role": "user", "content": prompt},
        ],
    )

    return resp.choices[0].message.content


# =========================
# 5) DEMO
# =========================
print("\n🤖 질문 1")
print(
    explain_with_sources(
        "손해배상 한도 조항에서 어떤 내용이 규정돼 있어?",
        namespace="contracts"
    )
)

print("\n🤖 질문 2")
print(
    explain_with_sources(
        "그 예외 조항은 어떤 의미야?",
        namespace="contracts"
    )
)

✅ Pinecone 기반 자료 해설형 챗봇 준비 완료

🤖 질문 1
1. [자료 발췌] : "본 계약의 손해배상 한도는 연간 계약 금액의 100%로 한다. 단, 고의 또는 중과실, 비밀유지 위반, 지식재산권 침해의 경우 손해배상 한도를 적용하지 않는다. 계약 해지는 30일 전 서면 통지로 가능하다."

2. [자료 정리] : 손해배상의 최대 한도는 연간 계약 금액의 100%로 제한되며, 그러나 고의, 중과실, 비밀유지 위반, 지식재산권 침해 등 특정 경우에는 이 한도가 적용되지 않는다. 또한 계약 해지는 30일 전에 서면으로 통지해야 한다.

3. [해설] : 계약에 따라 손해배상은 연간 계약 금액만큼 최대한 제한되지만, 고의나 중요한 법 위반 시에는 무제한 배상이 가능함을 명시하여 기업 간 책임 범위를 구체적으로 규정하고 있다.

🤖 질문 2
1. [자료 발췌] : "본 계약의 손해배상 한도는 연간 계약 금액의 100%로 한다. 단, 고의 또는 중과실, 비밀유지 위반, 지식재산권 침해의 경우 손해배상 한도를 적용하지 않는다. 계약 해지는 30일 전 서면 통지로 가능하다."

2. [자료 정리] : 계약에서는 손해배상 한도를 연간 계약 금액의 100%로 정하고 있지만, 고의나 중과실, 비밀유지 위반, 지식재산권 침해와 같은 특정한 경우에는 이 한도를 적용하지 않는 예외 조항이 있다.

3. [해설] : 이 예외 조항은 계약 당사자가 고의적이거나 중대한 과실행위, 비밀정보 누설, 또는 지식재산권을 침해했을 때는 손해배상의 한도에서 벗어나 더 큰 배상을 요구할 수 있음을 의미한다.


In [19]:
import os
import re
from dotenv import load_dotenv
from openai import OpenAI
from pinecone import Pinecone

# =========================
# 0) ENV & CLIENT
# =========================
load_dotenv(override=True)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")

if not OPENAI_API_KEY or not PINECONE_API_KEY or not PINECONE_INDEX_NAME:
    raise ValueError("❌ .env에 OPENAI_API_KEY / PINECONE_API_KEY / PINECONE_INDEX_NAME 설정 필요")

client = OpenAI(api_key=OPENAI_API_KEY)
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(PINECONE_INDEX_NAME)

EMBED_MODEL = "text-embedding-3-large"
CHAT_MODEL  = "gpt-4.1-mini"

DEFAULT_NS = "__default__"  # ✅ 네 stats 기준 실제 데이터가 있는 곳

print("✅ 법령 근거 해설 챗봇 준비 완료")
print("   - Index:", PINECONE_INDEX_NAME)
print("   - Namespace:", DEFAULT_NS)

# =========================
# 1) EMBEDDING
# =========================
def embed(text: str) -> list[float]:
    return client.embeddings.create(model=EMBED_MODEL, input=text).data[0].embedding

# =========================
# 2) Safe KR sentence split (no variable look-behind)
# =========================
def split_sentences_kr(text: str) -> list[str]:
    text = (text or "").strip()
    if not text:
        return []

    # 1) .,?,! 뒤 공백 분리
    parts = re.split(r'(?<=[\.\?\!])\s+', text)

    # 2) '다.' 기준 추가 분리(look-behind 없이)
    final = []
    for p in parts:
        sub = re.split(r'(다\.)\s*', p)
        buf = ""
        i = 0
        while i < len(sub):
            token = sub[i]
            if token == "다.":
                buf += token
                if buf.strip():
                    final.append(buf.strip())
                buf = ""
            else:
                buf += token
            i += 1
        if buf.strip():
            final.append(buf.strip())

    # 정리
    final = [s.strip() for s in final if s and s.strip()]
    return final

TEXT_KEYS = ["text", "page_content", "content", "body", "chunk", "excerpt", "paragraph", "raw_text"]

def extract_text(md: dict) -> str:
    if not isinstance(md, dict):
        return ""
    for k in TEXT_KEYS:
        v = md.get(k)
        if isinstance(v, str) and v.strip():
            return v.strip()
    data = md.get("data")
    if isinstance(data, dict):
        for k in TEXT_KEYS:
            v = data.get(k)
            if isinstance(v, str) and v.strip():
                return v.strip()
    return ""

def is_law_like(md: dict, sent: str) -> bool:
    law_type = str(md.get("law_type") or "").lower()
    category = str(md.get("category") or "").lower()
    source   = str(md.get("source") or "").lower()
    meta_blob = " ".join([law_type, category, source])

    if any(x in meta_blob for x in ["law", "법령", "조문", "statute", "regulation", "민법", "주택임대차"]):
        return True
    if re.search(r"\b제\s*\d+\s*조\b", sent):
        return True
    if any(k in sent for k in ["민법", "주택임대차", "조", "시행령", "시행규칙", "법률"]):
        return True
    return False

# =========================
# 3) Pinecone에서 법령 문장 뽑기
# =========================
def fetch_law_quotes(query: str, namespace: str = DEFAULT_NS, top_k: int = 30, max_quotes: int = 8):
    qvec = embed(query)
    res = index.query(
        vector=qvec,
        top_k=top_k,
        include_metadata=True,
        namespace=namespace,
    )

    pool = []
    for m in res.matches or []:
        md = m.metadata or {}
        raw = extract_text(md)
        if not raw:
            continue

        for sent in split_sentences_kr(raw):
            pool.append({
                "score": float(getattr(m, "score", 0.0)),
                "doc_id": md.get("doc_id") or md.get("source") or "unknown",
                "chunk_id": md.get("chunk_id"),
                "law_type": md.get("law_type") or md.get("category") or "",
                "text": sent,
                "law_like": is_law_like(md, sent),
            })

    if not pool:
        return []

    # dedupe
    seen = set()
    cleaned = []
    for s in pool:
        norm = re.sub(r"\s+", "", s["text"])
        if len(s["text"]) < 10:
            continue
        if norm in seen:
            continue
        seen.add(norm)
        cleaned.append(s)

    cleaned.sort(key=lambda x: (1 if x["law_like"] else 0, x["score"]), reverse=True)
    return cleaned[:max_quotes]

# =========================
# 4) LLM: 발췌 → 요지 → 해설
# =========================
SYSTEM = """너는 '법령 근거 해설' 챗봇이다.

절대 규칙:
- 반드시 제공된 [법령 발췌] 문장만을 근거로 답한다.
- 발췌에 없는 사실/법리/절차/판례/추정/일반론을 절대 추가하지 마라.
- 불확실하거나 발췌만으로 답을 확정할 수 없으면 '근거 부족' 또는 '추가 확인 필요'라고 말하라.

[출력 형식]
1) [법령 발췌] : 4~8개 문장을 따옴표로 그대로 인용하고 (doc/chunk) 표기
2) [조문 요지] : 발췌의 의미를 중복 없이 재구성(새 정보 금지)
3) [해설] : 요지를 쉬운 말로 설명(새 정보 금지)
4) [추가로 확인할 사실] : 필요한 질문 2~5개
"""

def law_chat(question: str, namespace: str = DEFAULT_NS, top_k: int = 30, max_quotes: int = 8):
    quotes = fetch_law_quotes(question, namespace=namespace, top_k=top_k, max_quotes=max_quotes)
    if not quotes:
        return "근거 부족"

    quote_block = "\n".join(
        [f'- "{q["text"]}" (doc={q["doc_id"]}, chunk={q["chunk_id"]})' for q in quotes]
    )

    prompt = f"""
[질문]
{question}

[법령 발췌]
{quote_block}
"""

    resp = client.chat.completions.create(
        model=CHAT_MODEL,
        messages=[
            {"role": "system", "content": SYSTEM},
            {"role": "user", "content": prompt},
        ],
    )
    return resp.choices[0].message.content

# =========================
# 5) Demo
# =========================
print("\n🤖 데모")
print(law_chat("전세 계약에서 대항력과 확정일자, 우선변제에 관한 법령 근거를 발췌해서 해설해줘"))

✅ 법령 근거 해설 챗봇 준비 완료
   - Index: realestate
   - Namespace: __default__

🤖 데모
1) [법령 발췌] : "주택임대차보호법 :주택임대차보호법은 주거용 건물의 임대차에 관하여 민법에대한 특례를 규정함으로써 임대인과 임차인을 보호하고 국민주거생활의 안정을 보장함을목적으로 하여 제정된 법률입니다(1981. 법률 제3379호). 이 법률은 사실상 주택에대한 채권적 전세나 임대차관계를 규정하고 있는 것으로, 임차인의 권리를 보호하기 위한중요한 법률입니다." (doc=전세피해법률상담사례집_250102.pdf, chunk=case_1_29)

2) [조문 요지] : 주택임대차보호법은 주거용 건물 임대차에 대해 민법의 특례를 두어 임대인과 임차인을 보호하고 국민 주거생활의 안정을 보장하기 위해 제정된 법률이며, 주택의 전세 또는 임대차관계에서 임차인의 권리를 보호하는 중요한 법률이다.

3) [해설] : 주택임대차보호법은 집을 빌려주는 사람과 빌리는 사람 모두를 보호하기 위해 만들어진 법이에요. 특히 전세 같은 주택 임대차 관계에서 세입자를 지켜주기 위한 중요한 법이에요.

4) [추가로 확인할 사실] : 
- 주택임대차보호법에서 대항력과 확정일자에 관한 구체적인 규정은 무엇인가요?
- 전세권자의 우선변제권에 관한 법률 조항은 어디에 있나요?
- 대항력 발생 시점과 확정일자의 관계에 대해 좀 더 구체적으로 설명할 법령 내용이 있나요?
- 임대차 계약에서 확정일자 부여 절차는 어떻게 되나요?


In [20]:
qvec = embed("대항력 확정일자 우선변제")
res = index.query(
    vector=qvec,
    top_k=8,
    include_metadata=True,
    namespace="__default__"
)

for i, m in enumerate(res.matches or [], 1):
    md = m.metadata or {}
    print(f"\n[{i}] score={m.score:.3f}")
    print("law_type:", md.get("law_type"), "category:", md.get("category"), "source:", md.get("source"))
    print("text:", (md.get("text") or "")[:200])


[1] score=0.432
law_type: 주택임대차보호법 category: law source: 주택임대차보호법(법률)(제21065호)(20260102).docx
text: 임차인이 대항력이나 우선변제권을 갖추고 「민법」

[2] score=0.432
law_type: 주택임대차보호법 category: law source: 주택임대차보호법(법률)(제21065호)(20260102).docx
text: 임차인이 대항력이나 우선변제권을 갖추고 「민법」

[3] score=0.403
law_type: 규칙 category: rule source: 주택임대차계약증서의 확정일자 부여 및 정보제공에 관한 규칙(대법원규칙)(제02986호)(20210610).docx
text: 제1항의 확정일자정보는 20년간 보존한다.

[4] score=0.400
law_type: 규칙 category: rule source: 주택임대차계약증서의 확정일자 부여 및 정보제공에 관한 규칙(대법원규칙)(제02986호)(20210610).docx
text: 주택임대차계약증서의 확정일자 부여 및 정보제공에 관한 규칙 일부를 다음과 같이 개정한다.

[5] score=0.400
law_type: 시행규칙/대법원규칙 category: rule source: 주택임대차계약증서의 확정일자 부여 및 정보제공에 관한 규칙(대법원규칙)(제02986호)(20210610).docx
text: 주택임대차계약증서의 확정일자 부여 및 정보제공에 관한 규칙 일부를 다음과 같이 개정한다.

[6] score=0.385
law_type: 주택임대차보호법 category: law source: 주택임대차보호법(법률)(제21065호)(20260102).docx
text: 제1항에 따라 우선변제를 받을 임차인 및 보증금 중 일정액의 범위와 기준은

[7] score=0.385
law_type: 주택임대차보호법 category: law source: 주택임대차보호법(법률)(제21065호)(20260102).docx
text: